In [1]:
#RUN
from pandas import DataFrame
import pandas as pd
import numpy as np

In [2]:
#RUN
try:
    from urllib import urlencode
except ImportError:
    from urllib.parse import urlencode
    
from jsonapi_client import Session, Filter
import pycurl

API_BASE = 'https://www.ebi.ac.uk/metagenomics/api/latest/'

In [23]:
#RUN
# 'studies' bisa diganti dengan target semisal 'sample' ; 'MGYS00001939' merupakan accession
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000858').resource

In [24]:
#RUN
#memunculkan study_name
std

<ResourceObject: studies: MGYS00000858 (2024379674072) (2024379674072)>

In [25]:
#RUN
#melihat jumlah sample pada study di atas
std.samples_count

7

In [26]:
#RUN/SKIP
sample = std.samples
#melihat informasi pada salah satu sample yang dipilih
x=sample[0]
x.sample_metadata

[{'key': 'investigation type', 'unit': None, 'value': 'metagenome'},
 {'key': 'project name',
  'unit': None,
  'value': 'Monitoring bacterial community changes during leaf litter degradation of Zea mays in arable soil'},
 {'key': 'geographic location (longitude)',
  'unit': None,
  'value': '48.5031 11.4468'},
 {'key': 'geographic location (depth)', 'unit': None, 'value': '0.05 m'},
 {'key': 'geographic location (country and/or sea,region)',
  'unit': None,
  'value': 'Scheyern, Germany'},
 {'key': 'collection date', 'unit': None, 'value': 'Nov-2009'},
 {'key': 'environment (biome)', 'unit': None, 'value': 'leaf litter'},
 {'key': 'environment (feature)', 'unit': None, 'value': 'arable soil'},
 {'key': 'environment (material)', 'unit': None, 'value': 'leaves'},
 {'key': 'environmental package',
  'unit': None,
  'value': 'MIGS/MIMS/MIMARKS.soil'},
 {'key': 'elevation', 'unit': None, 'value': '450 m'},
 {'key': 'pH', 'unit': None, 'value': '6'},
 {'key': 'history/agrochemical additions

In [ ]:
#SKIP
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

biome_label = 'environment (biome)'
elev_label = 'elevation'
df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
df.index.name = 'ID Sample'

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        df.loc[sample.accession] = [
            sample.sample_name,
            sample.longitude,
            sample.latitude,
            get_metadata(sample.sample_metadata, elev_label),
            get_metadata(sample.sample_metadata, biome_label),
            sample.geo_loc_name
        ]
df

In [37]:
#RUN
def get_metadata(metadata, key):
    import html
    for m in metadata:
        if m['key'].lower() == key.lower():
            value = m['value']
            unit = html.unescape(m['unit']) if m['unit'] else ""
            return "{value} {unit}".format(value=value, unit=unit)
    return None

#biome_label = 'environment (biome)'
#elev_label = 'elevation'
#df = DataFrame(columns=('Sample Name', 'Longitude', 'Latitude', 'Elevation', 'Biome', 'Country'))
#df.index.name = 'ID Sample'
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []

with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00000858',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        acc.append(sample.accession)
        sample_name.append(sample.sample_name)
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))
        analyze.append(b.accession)
        meta = sample.sample_metadata
        meta = pd.DataFrame.from_dict(meta)
        meta = meta.set_index('key') 
        meta = meta.drop(columns='unit')
        meta = meta.transpose()
        df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

analyses: MGYA00070106
analyses: MGYA00070102
analyses: MGYA00070117
analyses: MGYA00070079
analyses: MGYA00070084
analyses: MGYA00070114
analyses: MGYA00070098
analyses: MGYA00070108
analyses: MGYA00070077
analyses: MGYA00070109
analyses: MGYA00070082
analyses: MGYA00070095
analyses: MGYA00070111
analyses: MGYA00070112
analyses: MGYA00070087
analyses: MGYA00070097
analyses: MGYA00070085
analyses: MGYA00070090
analyses: MGYA00070107
analyses: MGYA00070103
analyses: MGYA00070080
analyses: MGYA00070089
analyses: MGYA00070086
analyses: MGYA00070110
analyses: MGYA00070088
analyses: MGYA00070078
analyses: MGYA00070118
analyses: MGYA00070091
analyses: MGYA00070101
analyses: MGYA00070115
analyses: MGYA00070116
analyses: MGYA00070100
analyses: MGYA00070113
analyses: MGYA00070105
analyses: MGYA00070094
analyses: MGYA00070104
analyses: MGYA00070083
analyses: MGYA00070093
analyses: MGYA00070099
analyses: MGYA00070081
analyses: MGYA00070092
analyses: MGYA00070096


In [38]:
df

key,investigation type,project name,geographic location (longitude),geographic location (depth),"geographic location (country and/or sea,region)",collection date,environment (biome),environment (feature),environment (material),environmental package,...,current land use,current vegetation,horizon,sample weight for DNA extraction,composite design/sieving (if any),soil type,history/tillage,NCBI sample classification,instrument model,BioSampleModel
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.05 m,"Scheyern, Germany",Nov-2009,leaf litter,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.05 m,"Scheyern, Germany",Dec-2009,leaf litter,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.05 m,"Scheyern, Germany",Oct-2009,leaf litter,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.00 m,"Scheyern, Germany",Oct-2009,fresh leaves,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.40 m,"Scheyern, Germany",Dec-2009,leaf litter,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.40 m,"Scheyern, Germany",Nov-2009,leaf litter,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
value,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.40 m,"Scheyern, Germany",Oct-2009,leaf litter,arable soil,leaves,MIGS/MIMS/MIMARKS.soil,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"


In [ ]:
#SKIP
with Session(API_BASE) as s:
    params = {
        'study_accession': 'MGYS00003963',
        'page_size': 100,
    }
    f = Filter(urlencode(params))
    for sample in s.iterate('samples', f):
        for a in sample.runs:
        #print('runs: '+str(a.accession))
            for b in a.analyses:
                print('analyses: '+str(b.accession))

In [ ]:
#RUN/SKIP
#memunculkan list sample sesuai jumlah
std.samples

In [ ]:
#SKIP
blank = []
for x in sample:
    #print(x.sample_metadata)
    df = pd.DataFrame(x.sample_metadata)
    df['key'].tolist()
    print(df)
    #df = pd.DataFrame(x.sample_metadata)
    #print(df)
    #index = np.where(df['key'].str.contains('location'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('biome'))[0]
    #blank.append(df.iloc[index, :])
    #index = np.where(df['key'].str.contains('elevation'))[0]
    #blank.append(df.iloc[index, :])
    #df_new = pd.concat(blank)
#df_new

In [ ]:
#RUN
with Session(API_BASE) as s:
    std = s.get('studies', 'MGYS00000837').resource
print(std.accession)
#melihat informasi pada salah satu sample yang dipilih
#sample = std.samples

In [ ]:
#SKIP
for x in sample:
    print('sample: '+str(x.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#membuat container kosong di bawah :
df = pd.DataFrame([])
acc = []
sample_name = []
analyze = []
for x in sample:
    acc.append(x.accession)
    sample_name.append(x.sample_name)
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
    analyze.append(b.accession)
    meta = x.sample_metadata
    meta = pd.DataFrame.from_dict(meta)
    meta = meta.set_index('key') 
    meta = meta.drop(columns='unit')
    meta = meta.transpose()
    df = df.append(meta)
acc = pd.DataFrame(acc, columns=['ID_Sample'])
sample_name = pd.DataFrame(sample_name, columns=['Sample Name'])
analyze = pd.DataFrame(analyze, columns=['ID_Analyses'])

In [40]:
#RUN
#reset index agar dapat di merge
df = df.reset_index()
df = df.drop(columns='index')
#df = df.drop(columns='level_0')

In [41]:
#RUN
#script merge
merged = acc.merge(sample_name, left_index=True, right_index=True, how='inner')
merged = merged.merge(analyze, left_index=True, right_index=True, how='inner')
merged = merged.merge(df, left_index=True, right_index=True, how='inner')
#df
merged

,ID_Sample,Sample Name,ID_Analyses,investigation type,project name,geographic location (longitude),geographic location (depth),"geographic location (country and/or sea,region)",collection date,environment (biome),...,current land use,current vegetation,horizon,sample weight for DNA extraction,composite design/sieving (if any),soil type,history/tillage,NCBI sample classification,instrument model,BioSampleModel
0,SRS541549,4weeks_minTillage,MGYA00070114,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.05 m,"Scheyern, Germany",Nov-2009,leaf litter,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
1,SRS541551,8weeks_minTillage,MGYA00070095,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.05 m,"Scheyern, Germany",Dec-2009,leaf litter,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
2,SRS541547,2weeks_minTillage,MGYA00070090,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.05 m,"Scheyern, Germany",Oct-2009,leaf litter,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
3,SRS527315,0weeks_fresh,MGYA00070110,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.00 m,"Scheyern, Germany",Oct-2009,fresh leaves,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
4,SRS541550,8weeks_convTillage,MGYA00070115,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.40 m,"Scheyern, Germany",Dec-2009,leaf litter,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
5,SRS541548,4weeks_convTillage,MGYA00070104,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.40 m,"Scheyern, Germany",Nov-2009,leaf litter,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"
6,SRS541546,2weeks_convTillage,MGYA00070096,metagenome,Monitoring bacterial community changes during ...,48.5031 11.4468,0.40 m,"Scheyern, Germany",Oct-2009,leaf litter,...,farming,Zea mays,B,0.5 g,2 mm,cambisol,disc plough,410658,454 GS FLX Titanium,"MIMS.me,MIGS/MIMS/MIMARKS.soil"


In [ ]:
#RUN SEKALI AJA
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)

In [42]:
#RUN
#masuk ke folder "Study_Jagung"
%cd Study_Jagung
#export ke .csv
merged.to_csv(std.accession+'.csv')

[WinError 2] The system cannot find the file specified: 'Study_Jagung'
D:\Seminar\Seminar_Anam\Study_Jagung


In [ ]:
#SKIP
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))

In [ ]:
#SKIP
#memasukkan sample yang dipilih ke bentuk kolom
df = pd.DataFrame(sample[0].sample_metadata)
df

In [ ]:
#SKIP
blank = []

In [ ]:
#SKIP
#membuat index baru dari list 'key' yang dipilih
index = np.where(df['key'].str.contains('location'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('biome'))[0]
blank.append(df.iloc[index, :])
index = np.where(df['key'].str.contains('elevation'))[0]
blank.append(df.iloc[index, :])

In [ ]:
#SKIP
#'key' yang dilih dijadikan data frame baru
df_new = pd.concat(blank)
df_new

In [ ]:
#SKIP
filename_sample = '' 

In [ ]:
#SKIP
#membuat loop accession di dalam bentuk list
for x in sample:
    #print('sample: '+str(i.accession))
    for a in x.runs:
        #print('runs: '+str(a.accession))
        for b in a.analyses:
            print('analyses: '+str(b.accession))
            #for z in y.downloads:
                #if 'json.biom' in z.url:
                    #print(z.url)

In [ ]:
#SKIP
#membuat folder
from pathlib import Path
import os
_dir = os.getcwd()
#_dir = 'D:\\Seminar\\Seminar_Anam'
_dir = os.path.join(_dir, 'Study_Jagung')
if not os.path.exists(_dir):
    os.makedirs(_dir)
print(_dir)
#%rmdir(Study+str)
#with open('_dir'+i.accession+'.biom', 'wb') as f:
    #c = pycurl.Curl()
    #c.setopt(c.URL, z.url)
    #c.setopt(c.WRITEDATA, f)
    #c.perform()
#c.close()